In [5]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [16]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

from sklearn.model_selection import train_test_split

In [4]:
##GPU 사용
device = torch.device("cuda:0")

In [6]:
bert, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [77]:
dialogue = pd.read_excel('/content/drive/MyDrive/해커톤/감성대화말뭉치(최종데이터)_Training.xlsx')

In [78]:
print(len(dialogue))
dialogue.head()

40879


,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,50882,청소년,여성,학업 및 진로,해당없음,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,31303,노년,남성,재정,만성질환 무,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,47200,노년,여성,재정,만성질환 유,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN


In [79]:
dialogue['감정_대분류'].unique() # '기쁨 ', '불안 ' -> 공백포함 데이터 있음

array(['기쁨', '불안', '당황', '슬픔', '분노', '상처', '불안 ', '기쁨 '], dtype=object)

In [80]:
dialogue.loc[(dialogue['감정_대분류'] == "기쁨"), '감정'] = 0
dialogue.loc[(dialogue['감정_대분류'] == '기쁨 '), '감정'] = 0
dialogue.loc[(dialogue['감정_대분류'] == "불안"), '감정'] = 1
dialogue.loc[(dialogue['감정_대분류'] == "불안 "), '감정'] = 1 
dialogue.loc[(dialogue['감정_대분류'] == "당황"), '감정'] = 2 
dialogue.loc[(dialogue['감정_대분류'] == "슬픔"), '감정'] = 3 
dialogue.loc[(dialogue['감정_대분류'] == "분노"), '감정'] = 4 
dialogue.loc[(dialogue['감정_대분류'] == "상처"), '감정'] = 5 


In [81]:
#결측치
dialogue.isna().sum()

번호            0
연령            0
성별            0
상황키워드         0
신체질환          0
감정_대분류        0
감정_소분류        0
사람문장1         0
시스템응답1        0
사람문장2         0
시스템응답2        0
사람문장3      9820
시스템응답3     9820
사람문장4     39407
시스템응답4    39517
감정            0
dtype: int64

In [82]:
dialogue = dialogue.loc[:,['감정_대분류','사람문장1','감정']]
dialogue.dropna(inplace=True)
dialogue.isna().sum()    

감정_대분류    0
사람문장1     0
감정        0
dtype: int64

In [83]:
data_list = []
for q, label in zip(dialogue['사람문장1'], dialogue['감정'])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

In [84]:
print(len(data_list))
print(data_list[0])
print(data_list[-1])

40879
['아내가 드디어 출산하게 되어서 정말 신이 나.', '0']
['친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아.', '1']


In [85]:
x_train, x_test = train_test_split(data_list, test_size=0.20, random_state=42)
print(len(x_train))
print(len(x_test))
print(x_train[0])
print(x_test[0])

32703
8176
['회사 사람들 때문에 너무 구역질 나.', '4']
['도서관 가면 조용해서 그런지 항상 집중이 잘되는 거 같아. 오늘 공부 많이 해서 너무 행복해.', '0']


In [86]:
#학습모델
class SKT_BERT(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [87]:
## 파라미터
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [88]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [89]:
x_train = SKT_BERT(x_train, 0, 1, tok, max_len, True, False)
x_test = SKT_BERT(x_test, 0, 1, tok, max_len, True, False)

In [90]:
train_dataloader = torch.utils.data.DataLoader(x_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(x_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [91]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [92]:
model = BERTClassifier(bert,dr_rate=0.5).to(device)

In [93]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [75]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

In [94]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/511 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.992851972579956 train acc 0.109375
epoch 1 batch id 201 loss 0.8073935508728027 train acc 0.6401585820895522
epoch 1 batch id 401 loss 0.9551836848258972 train acc 0.6477556109725686
epoch 1 train acc 0.6479237528344671


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/128 [00:00<?, ?it/s]

...

epoch 9 test acc 0.6487223307291666


  0%|          | 0/511 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.18964175879955292 train acc 0.96875
epoch 10 batch id 201 loss 0.18379563093185425 train acc 0.9448072139303483
epoch 10 batch id 401 loss 0.24014802277088165 train acc 0.9422927057356608
epoch 10 train acc 0.9389963462553972


  0%|          | 0/128 [00:00<?, ?it/s]

epoch 10 test acc 0.646240234375


In [96]:
#모델 저장
import os
os.chdir('/content/drive/MyDrive/해커톤/models/')
os.getcwd()

'/content/drive/MyDrive/해커톤/models'

In [97]:
path = '/content/drive/MyDrive/해커톤/models/'
torch.save(model, path + 'SKT_BERT_model.pt')  # 전체 모델 저장
torch.save(model.state_dict(), 'SKT_BERT_model_state_dict.pt')  # 모델 객체의 state_dict 저장

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'SKT_BERT_model.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

In [98]:
#★★★현재경로가 model이 있는 폴더여야함★★★
import os
os.chdir('/content/drive/MyDrive/해커톤/models/')

model1 = torch.load('SKT_BERT_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('SKT_BERT_model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('SKT_BERT_model.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

In [99]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

/content/drive/MyDrive/해커톤/models/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [100]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = SKT_BERT(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁜")
            elif np.argmax(logits) == 1:
                test_eval.append("불안한")
            elif np.argmax(logits) == 2:
                test_eval.append("당황한")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픈")
            elif np.argmax(logits) == 4:
                test_eval.append("분노에 찬")
            elif np.argmax(logits) == 5:
                test_eval.append("상처받은")

        print(test_eval[0] + "감정이 느껴집니다.")

In [104]:
#질문 무한반복하기!
end = 1
while end == 1 :
    sentence = input("말을 입력해주세요 (종료: 0 입력) : ")
    if sentence == '0' :
        print('종료되었습니다.')
        break
    predict(sentence)
    print("\n")

말을 입력해주세요 (종료: 0 입력) : 갈 길이 먼데 빈차가 없네
슬픈감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 나한테 왜 그랬어요
불안한감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 넌 나에게 모욕감을 줬어
당황한감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 나도 그 애처럼 좋아요 좀 눌러줘
당황한감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 나도 너랑 말 좀 할 수 있게 해줘
분노에 찬감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 넘 넘 스윗한 너 정말 달콤한 걸
기쁜감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 너 왜 눈을 그렇게 떠?
슬픈감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 안녕하세요 적당히 바람이 시원해 기분이 너무 좋아요 유후 
기쁜감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 끝내줬어요 긴장한 탓에 엉뚱한 얘기만 늘어놓았죠 바보같이
슬픈감정이 느껴집니다.


말을 입력해주세요 (종료: 0 입력) : 0
종료되었습니다.
